# Data Preprocessing and Linear Modelling- Sklearn

In [53]:
# File importing

import pandas as pd
from AdvancedAnalytics import ReplaceImputeEncode
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [31]:
diamond=pd.read_excel('diamondswmissing.xlsx')

In [46]:
diamond.head()

,obs,price,Carat,cut,color,clarity,depth,table,x,y,z
0,1,326,0.798309,Ideal,E,SI2,61.5,55.0,3.95,3.980000,2.43000
1,2,326,0.210000,Premium,E,SI1,59.8,61.0,3.89,3.840000,2.31000
2,3,327,0.230000,Good,E,VS1,56.9,65.0,4.05,4.070000,2.31000
3,4,334,0.290000,Premium,I,VS2,62.4,58.0,4.20,5.736222,2.63000
4,5,335,0.310000,Good,J,SI2,63.3,58.0,4.34,4.350000,3.53902


In [36]:
# Function to get the count of elements in a column

def count(y):
    a=diamond[y][diamond[y].notnull()].unique()
    b=pd.DataFrame(index=['count'], columns=a)
    for word in a:
        b[word]=diamond[y].apply(lambda x: 1 if x==word else 0).sum()
    b['missing']=diamond[y].isnull().sum()
    return b
    

In [37]:
# Step 1 : Metadata preparation

data_map={\
            'cut':[3, ('Ideal', 'Premium', 'Good', 'Very Good', 'Fair'),[0,0]], \
            'color':[2,('E', 'I', 'J', 'H', 'F', 'G', 'D'),[0,0]],  \
            'clarity':[2,('SI2', 'SI1', 'VS1', 'VS2', 'VVS2', 'VVS1', 'I1', 'IF'),[0,0]], \
            'Carat':[0, (0,6),[0,0]], \
            'depth':[0,(43, 79), [0,0]], \
            'table':[0,(43,95), [0,0]], \
            'x':[0, (0,20), [0,0]], \
            'y':[0, (0,60), [0,0]], \
            'z':[0, (0,35), [0,0]]}

In [38]:
# Step 2 : Outlier analysis

outlier_analysis=ReplaceImputeEncode(data_map=data_map, display=True)
diamond_outlier_analysis=outlier_analysis.fit_transform(diamond)


********** Data Preprocessing ***********
Features Dictionary Contains:
0 Interval, 
0 Binary, 
0 Nominal, and 
11 Excluded Attribute(s).

Data contains 53940 observations & 11 columns.


Attribute Counts
.............. Missing  Outliers
cut......      2664         0
color....      1571         0
clarity..      1689         0
Carat....      5467         0
depth....      1653         0
table....      1600         0
x........      1598         0
y........      1636         0
z........      1639         0


In [39]:
# Step 3: Interval variable data imputation

interval_attributes=['Carat','depth','table','x','y','z']
interval_data=diamond.as_matrix(columns=interval_attributes)
interval_impute=preprocessing.Imputer(strategy='mean')
imputed_interval_data=interval_impute.fit_transform(interval_data)
diamond[interval_attributes]=imputed_interval_data

C:\Users\NITESH\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [40]:
# Step 3: Categorical variable data imputation

# Categorical variables require integer mapping before imputation

cat_map_cut={'Ideal':1, 'Premium':2, 'Good':3, 'Very Good':4, 'Fair':5}
cat_map_color={'E':1, 'I':2, 'J':3, 'H':4, 'F':5, 'G':6, 'D':8}
cat_map_clarity={'SI2':1, 'SI1':2, 'VS1':3, 'VS2':4, 'VVS2':5, 'VVS1':6, 'I1':7, 'IF':8}
diamond['cut']=diamond['cut'].map(cat_map_cut)
diamond['color']=diamond['color'].map(cat_map_color)
diamond['clarity']=diamond['clarity'].map(cat_map_clarity)

ordinal_nominal_attributes=['cut','color','clarity']
ordinal_nominal_matrix=diamond.as_matrix(columns=ordinal_nominal_attributes)
ordinal_nominal_impute=preprocessing.Imputer(strategy='most_frequent')
ordinal_nominal_imputed_data=ordinal_nominal_impute.fit_transform(ordinal_nominal_matrix)
diamond[ordinal_nominal_attributes]=ordinal_nominal_imputed_data


diamond['cut']=diamond['cut'].map({1:'Ideal', 2:'Premium', 3:'Good', 4:'Very Good', 5:'Fair'})
diamond['color']=diamond['color'].map({1:'E', 2:'I', 3:'J', 4:'H', 5:'F', 6:'G', 8:'D'})
diamond['clarity']=diamond['clarity'].map({1:'SI2', 2:'SI1', 3:'VS1', 4:'VS2', 5:'VVS2', 6:'VVS1', 7:'I1', 8:'IF'})

In [47]:
## After imputation

diamond.head()

,obs,price,Carat,cut,color,clarity,depth,table,x,y,z
0,1,326,0.798309,Ideal,E,SI2,61.5,55.0,3.95,3.980000,2.43000
1,2,326,0.210000,Premium,E,SI1,59.8,61.0,3.89,3.840000,2.31000
2,3,327,0.230000,Good,E,VS1,56.9,65.0,4.05,4.070000,2.31000
3,4,334,0.290000,Premium,I,VS2,62.4,58.0,4.20,5.736222,2.63000
4,5,335,0.310000,Good,J,SI2,63.3,58.0,4.34,4.350000,3.53902


In [49]:
# Step 4: One hot encoding of categorical features

def cat_encode(y):
    a=diamond[y][diamond[y].notnull()].unique()
    for word in a:
        diamond[word]=diamond[y].apply(lambda x: 1 if x==word else 0)
        
cat_encode('cut')
cat_encode('color')
cat_encode('clarity')   

diamond=diamond.drop(columns=['obs','cut','color','clarity'])

diamond.head()

,price,Carat,depth,table,x,y,z,Ideal,Premium,Good,...,F,D,SI2,SI1,VS1,VS2,VVS2,VVS1,I1,IF
0,326,0.798309,61.5,55.0,3.95,3.980000,2.43000,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,326,0.210000,59.8,61.0,3.89,3.840000,2.31000,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,327,0.230000,56.9,65.0,4.05,4.070000,2.31000,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,334,0.290000,62.4,58.0,4.20,5.736222,2.63000,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,335,0.310000,63.3,58.0,4.34,4.350000,3.53902,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [60]:
#Step 5: Modelling

# Feature and Target preparation

X_features=diamond.columns[diamond.columns!='price']
Y_features=['price']

X=diamond[X_features]
Y=diamond[Y_features]

# Train - Test Split
X_train, X_test,Y_train, Y_test=train_test_split(X,Y, test_size=0.3, random_state=101)

# Linear regression 
lm=LinearRegression()
lm.fit(X_train,Y_train,)

Predict_Xtrain=lm.predict(X_train)
Predict_Xtest=lm.predict(X_test)
Predict_X=lm.predict(X)

# Appending predicted price to original dataset

diamond['Predicted_price']=Predict_X

In [63]:
# Analysis of diamond observed price and predicted price

analysis=pd.DataFrame(index=['Observed_price','Predicted_price'], columns=['min','mean','max'])
analysis['min']=[diamond['price'].min(), diamond['Predicted_price'].min()]
analysis['mean']=[diamond['price'].mean(), diamond['Predicted_price'].mean()]
analysis['max']=[diamond['price'].max(), diamond['Predicted_price'].max()]

In [61]:
diamond.head()

,price,Carat,depth,table,x,y,z,Ideal,Premium,Good,...,D,SI2,SI1,VS1,VS2,VVS2,VVS1,I1,IF,Predicted_price
0,326,0.798309,61.5,55.0,3.95,3.980000,2.43000,1,0,0,...,0,1,0,0,0,0,0,0,0,-639.714434
1,326,0.210000,59.8,61.0,3.89,3.840000,2.31000,0,1,0,...,0,0,1,0,0,0,0,0,0,-2532.354288
2,327,0.230000,56.9,65.0,4.05,4.070000,2.31000,0,0,1,...,0,0,0,1,0,0,0,0,0,-1639.623217
3,334,0.290000,62.4,58.0,4.20,5.736222,2.63000,0,1,0,...,0,0,0,0,1,0,0,0,0,-739.284295
4,335,0.310000,63.3,58.0,4.34,4.350000,3.53902,0,0,1,...,0,1,0,0,0,0,0,0,0,-2982.069387


In [62]:
analysis

,min,mean,max
Observed_price,326.000000,3932.799722,18823.000000
Predicted_price,-9784.986787,3929.978788,41515.972481
